WE WILL WORK WITH BINARY DATASETS

Image preprocessing
(for black and white images)

In [ ]:
#Import important libraries
import tensorflow as tf
import os
import numpy as np
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

In [ ]:
#Set the paths to the directories for train and test images and masks
train_set_path = '/home/harshveer.singh/Downloads/drone_dataset/archive/binary_dataset/images/train_set/'
test_set_path = '/home/harshveer.singh/Downloads/drone_dataset/archive/binary_dataset/images/test_set/'

original_train_path = '/home/harshveer.singh/Downloads/drone_dataset/archive/binary_dataset/images/train_set/original_images/'
original_test_path = '/home/harshveer.singh/Downloads/drone_dataset/archive/binary_dataset/images/test_set/original_images/'
mask_train_path = '/home/harshveer.singh/Downloads/drone_dataset/archive/binary_dataset/images/train_set/images_semantic/'
mask_test_path = '/home/harshveer.singh/Downloads/drone_dataset/archive/binary_dataset/images/test_set/images_semantic/'

# List all files in the train_path directory
train_ids = [filename for filename in os.listdir(original_train_path) if os.path.isfile(os.path.join(original_train_path, filename))]

test_ids = [filename for filename in os.listdir(original_test_path) if os.path.isfile(os.path.join(original_test_path, filename))]

#The order might be jumbled
print("TRAIN PATHS :", train_ids)
print("TEST PATHS",test_ids)


np.random.seed = 42

#Dimentions to be used to resize all the images
img_width = 128
img_height = 128
img_channels = 3

#READ ALL THE IMAGES AND RESIZE THEM TO 128x128
x_train = np.zeros((len(train_ids), img_height,img_width,img_channels), dtype = np.uint8)
y_train = np.zeros((len(train_ids), img_height,img_width,1), dtype = np.bool_) #boolean - we want a yes or no answer


print("RESIZING TRAINING IMAGES")
for n,id_ in tqdm(enumerate(train_ids), total = len(train_ids)):   #tqdm just gives a progress bar
    
    path = original_train_path   #will give the path to the folder with the image
    img = imread(path +id_)[:,:,:img_channels]   #then go to the 'images' subfolder and add the image

    img = resize(img,(img_height,img_width), mode='constant',preserve_range=True)
    x_train[n] = img



print("RESIZING MASKS")
train_ids_masks= [filename for filename in os.listdir(mask_train_path) if os.path.isfile(os.path.join(mask_train_path, filename))]

test_ids_masks = [filename for filename in os.listdir(mask_test_path) if os.path.isfile(os.path.join(mask_test_path, filename))]


for n,id_ in tqdm(enumerate(train_ids_masks), total = len(train_ids)):   #tqdm just gives a progress bar
    
    path = mask_train_path   #will give the path to the folder fro the image
    masks_ = imread(path + id_)
    masks_ = np.expand_dims(resize(masks_,(img_height,img_width),mode='constant',preserve_range=True), axis =-1)
    y_train[n] = masks_



# print("X train data type ",x_train.dtype)
# print("Y train data type ",y_train.dtype)


x_test = np.zeros((len(test_ids), img_height,img_width,img_channels), dtype = np.uint8)

print("RESIZING testing IMAGES ")

for n,id_ in tqdm(enumerate(test_ids), total = len(test_ids)):   #tqdm just gives a progress bar

        path = original_test_path   #will give the path to the folder fro the image
        img = imread(path +id_)[:,:,:img_channels]   #then go to the 'images' subfolder and add the image


        img = resize(img,(img_height,img_width), mode='constant',preserve_range=True)
        x_test[n] = img

print("DONE!!!")


#UNCOMMENT TO PERFORM A SANITY CHECK
#image_x = random.randint(0,len(train_ids))
#imshow(x_train[image_x])
#plt.show()
#
#imshow(np.squeeze(y_train[image_x]))
#plt.show()
#
#print("Image ID",image_x)

Image preprocessing (for coloured binary images)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
# Define image dimensions
img_width = 128
img_height = 128
img_channels = 3

# Path to your dataset
image_path = '/home/harshveer.singh/Downloads/Kitti_dataset/data_road_224/training/image_2/'
mask_path = '/home/harshveer.singh/Downloads/Kitti_dataset/data_road_224/training/gt_image_2/'

train_ids = [filename for filename in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, filename))]
train_ids_masks= [filename for filename in os.listdir(mask_path) if os.path.isfile(os.path.join(mask_path, filename))]
# Function to load images and masks
def load_data(image_path, mask_path):
    images = []
    masks = []
    
    for n,id_ in tqdm(enumerate(train_ids), total = len(train_ids)): 
        path = image_path   #will give the path to the folder fro the image
        
        img = cv2.imread(path +id_)#[:,:,:img_channels]   #then go to the 'images' subfolder and add the image
        
        img = cv2.resize(img, (img_width, img_height))
        images.append(img)

        path = mask_path   #will give the path to the folder fro the image
        mask = cv2.imread(path +"gt_"+ id_)
        mask = cv2.resize(mask, (img_width, img_height))
        #My images are purple and yellow. Adjust the below value according to need
        road_mask = cv2.inRange(mask, (128, 0, 128), (128, 0, 128))  # Purple color range (background colour)
        road_mask = road_mask // 255  # Normalize to binary values (0 and 1)
        masks.append(road_mask)     
    
    return np.array(images), np.array(masks)

x_train, y_train = load_data(image_path, mask_path)

# Normalize the images
x_train = x_train / 255.0
y_train = np.expand_dims(y_train, axis=-1)  # Expand dimensions for masks

print(f"Images shape: {x_train.shape}")
print(f"Masks shape: {y_train.shape}")


Precdictions for the model


In [ ]:
model = tf.keras.models.load_model('./image_segmentation_model.h5')

ix = random.randint(0,len(x_train))

preds_train = model.predict(x_train[:int(x_train.shape[0]*0.9)],verbose=1)
preds_val = model.predict(x_train[int(x_train.shape[0]*0.9):],verbose=1)
preds_test = model.predict(x_test,verbose=1)

preds_train_t = (preds_train>0.5).astype(np.uint8)
preds_val_t = (preds_val>0.5).astype(np.uint8)
preds_test_t = (preds_test>0.5).astype(np.uint8)


#sanity check on some random train sample
ix = random.randint(0,len(preds_train_t))
imshow(x_train[ix])
plt.show()
imshow(np.squeeze(y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

#sanity check on some validation sample 
ix = random.randint(0,len(preds_val_t))
imshow(x_train[ix])
plt.show()
imshow(np.squeeze(y_train[ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Load your test images and ground truth masks
# Assuming x_test and y_test are already loaded

# Make predictions on the test set
preds_test = model.predict(x_test, verbose=1)

# Apply binary thresholding
#preds_test_t = (preds_test > 0.5).astype(np.uint8)
preds_test=preds_test.astype(np.uint8)
# Function to visualize the results
def visualize_results(x, y_pred, idx):
    plt.figure(figsize=(12, 4))
    
    # Original image
    plt.subplot(1, 3, 1)
    plt.title("Original Image")
    plt.imshow(x[idx])
    plt.axis('off')
    
    # # Ground truth mask (wont have these for test set)
    # plt.subplot(1, 3, 2)
    # plt.title("Ground Truth Mask")
    # plt.imshow(np.squeeze(y_true[idx]), cmap='gray')
    # plt.axis('off')
   
    
    # Predicted mask
    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(np.squeeze(y_pred[idx]),cmap='gray')
    
    plt.show()

# Visualize predictions for a few random test images
num_samples = 5  # Number of samples to visualize
for _ in range(num_samples):
    idx = random.randint(0, len(x_test) - 1)
    visualize_results(x_test, preds_test_t, idx)


# Download some random images and test
image1 = imread('/Downloads/image1.jpeg')[:,:,:img_channels]   
image1 = resize(image1,(img_height,img_width), mode='constant',preserve_range=True)

image2 = imread('/Downloads/image2.jpeg')[:,:,:img_channels]

image2 = resize(image2,(img_height,img_width), mode='constant',preserve_range=True)




random_test_images = np.zeros((2, img_height,img_width,img_channels), dtype = np.uint8)
random_test_images[0] = image1
random_test_images[1] = image2



print("New test images ....")
new_preds_test = model.predict(random_test_images, verbose=1)

for i in range(5):
    visualize_results(random_test_images, new_preds_test, i)
